In [1]:
import pandas as pd
import endpoints
from datetime import datetime, timedelta
# from config import SITE_EMAIL, SITE_PASSWORD
import time

In [3]:
today = datetime.today()
yesterday = today - timedelta(days=1)
today_str = today.strftime('%Y-%m-%d')
yesterday_str = yesterday.strftime('%Y-%m-%d')

In [ ]:
cookies_dict = endpoints.get_cookies(EMAIL, PASSWORD)
session = endpoints.get_session()
data = endpoints.get_l1_l2(yesterday_str, today_str, session, cookies_dict)

df = pd.json_normalize(data['data'])
df.rename({'id': 'l1_id', 'value': 'l1_name'}, axis=1, inplace=True)
df = df[['l1_id', 'l1_name', 'data']]\
    .explode('data')\
    .reset_index(drop=True)
df = df.drop(0)\
    .reset_index(drop=True)
df_expanded = pd.json_normalize(df['data'])
df = pd.concat([df[['l1_id', 'l1_name']], df_expanded], axis=1)\
    .rename({'id': 'l2_id', 'value': 'l2_name'}, axis=1)
df = df[['l1_id', 'l1_name', 'l2_id', 'l2_name']]

In [7]:
temp_dfs = []
for l2_id in df['l2_id']:
    data = endpoints.get_l3(yesterday_str, l2_id, today_str, session, cookies_dict)
    df_temp = pd.json_normalize(data['data'])
    df_temp['l2_id'] = l2_id
    temp_dfs.append(df_temp)
    time.sleep(1)

if temp_dfs:
    df_l3 = pd.concat(temp_dfs, ignore_index=True)
    df_l3 = df_l3.rename(columns={'id': 'l3_id', 'name': 'l3_name'})\
        .drop(['url', 'path', 'isFavorite'], axis=1)
else:
    df_l3 = pd.DataFrame()